In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, PowerTransformer
from keras import models
from keras import layers
import tensorflow as tf
from Informer import Informer
import warnings
warnings.filterwarnings('ignore')

In [27]:
def create_sequences(X, y, window_size):
    seq_X = []
    seq_y = []
    for i in range(len(X) - window_size):
        seq_X.append(X[i:i+window_size])
        seq_y.append(y[i+window_size])

    return np.array(seq_X), np.array(seq_y)

In [7]:
# read dataset
df= pd.read_csv('Data/Tetuan City power consumption.csv')

In [13]:
df.head()

,DateTime,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,Zone 1 Power Consumption,Zone 2 Power Consumption,Zone 3 Power Consumption
0,1/1/2017 0:00,6.559,73.8,0.083,0.051,0.119,34055.69620,16128.87538,20240.96386
1,1/1/2017 0:10,6.414,74.5,0.083,0.070,0.085,29814.68354,19375.07599,20131.08434
2,1/1/2017 0:20,6.313,74.5,0.080,0.062,0.100,29128.10127,19006.68693,19668.43373
3,1/1/2017 0:30,6.121,75.0,0.083,0.091,0.096,28228.86076,18361.09422,18899.27711
4,1/1/2017 0:40,5.921,75.7,0.081,0.048,0.085,27335.69620,17872.34043,18442.40964


In [ ]:
# create date dataset
df_date= df[['DateTime']]
df_date['DateTime']= pd.to_datetime(df['DateTime'])
df_date['minute']= df_date['DateTime'].dt.minute
df_date['hour']= df_date['DateTime'].dt.hour
df_date['weekday']= df_date['DateTime'].dt.weekday
df_date['month']= df_date['DateTime'].dt.month

array([[Timestamp('2017-01-01 00:00:00'), 0, 0, 6, 1],
       [Timestamp('2017-01-01 00:10:00'), 10, 0, 6, 1],
       [Timestamp('2017-01-01 00:20:00'), 20, 0, 6, 1],
       ...,
       [Timestamp('2017-12-30 23:30:00'), 30, 23, 5, 12],
       [Timestamp('2017-12-30 23:40:00'), 40, 23, 5, 12],
       [Timestamp('2017-12-30 23:50:00'), 50, 23, 5, 12]], dtype=object)

In [33]:
# create & prepare data&target dataset
df_data= df.drop(['DateTime', 'Zone 1 Power Consumption'], axis=1)
target= df[['Zone 1 Power Consumption']]

# power scaler
Power_scaler= PowerTransformer(method='box-cox')
df_data['normed_general diffuse flows'] = Power_scaler.fit_transform(df_data[['general diffuse flows']])
df_data['normed_diffuse flows'] = Power_scaler.fit_transform(df_data[['diffuse flows']])
df_data['normed_humidity'] = np.clip(df_data['Humidity'], a_min=40, a_max=90)
df_data = df_data.drop(['Humidity', 'general diffuse flows', 'diffuse flows'], axis=1)

# MinMax scaler
MinMax_scaler= MinMaxScaler(feature_range=(0, 1))
df_data= MinMax_scaler.fit_transform(df_data)
target= MinMax_scaler.fit_transform(target)

# sequencing data
data, target= create_sequences(df_data, target, window_size=144)

In [32]:
target

array([[0.39762262],
       [0.37414908],
       [0.35083415],
       ...,
       [0.40970272],
       [0.3931869 ],
       [0.37730631]])